In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [2]:
df = pd.read_csv('fy2020_Dallas_Animal_Shelter_Data.csv')

In [3]:
df.head()

,Animal Id,Animal Type,Animal Breed,Kennel Number,Kennel Status,Tag Type,Activity Number,Activity Sequence,Source Id,Census Tract,Council District,Intake Type,Intake Subtype,Intake Total,Reason,Staff Id,Intake Date,Intake Time,Due Out,Intake Condition,Hold Request,Outcome Type,Outcome Subtype,Outcome Date,Outcome Time,Receipt Number,Impound Number,Service Request Number,Outcome Condition,Chip Status,Animal Origin,Additional Information,Month,Year
0,A0144701,DOG,HAVANESE,VT 12,IMPOUNDED,NaN,NaN,1,P0098773,006301,4,OWNER SURRENDER,GENERAL,1,PERSNLISSU,CDM,11/08/2019,15:48:00,11/14/2019,APP SICK,NaN,RETURNED TO OWNER,WALK IN,11/09/2019,11:31:00,R19-558731,K19-486742,NaN,APP SICK,SCAN CHIP,OVER THE COUNTER,RETURNED TO OWNER,NOV.2019,FY2020
1,A0442587,DOG,TERRIER MIX,FREEZER,IMPOUNDED,NaN,NaN,1,P0492284,007102,2,OWNER SURRENDER,DEAD ON ARRIVAL,1,OTHRINTAKS,CDM,11/10/2019,14:18:00,11/10/2019,DEAD,NaN,DEAD ON ARRIVAL,DISPOSAL,11/10/2019,00:00:00,NaN,K19-486954,NaN,DEAD,SCAN CHIP,OVER THE COUNTER,NaN,NOV.2019,FY2020
2,A0458972,DOG,CATAHOULA,RECEIVING,UNAVAILABLE,NaN,A19-195601,1,P9991718,004600,1,STRAY,AT LARGE,1,OTHER,MG1718,10/03/2019,11:08:00,10/03/2019,TREATABLE REHABILITABLE NON-CONTAGIOUS,NaN,RETURNED TO OWNER,FIELD,10/03/2019,13:36:00,NaN,K19-482022,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN NO CHIP,SWEEP,NaN,OCT.2019,FY2020
3,A0525642,DOG,GERM SHEPHERD,INJD 001,IMPOUNDED,NaN,A19-196573,1,P0903792,016605,8,OWNER SURRENDER,GENERAL,1,OTHER,RA 1549,10/11/2019,09:55:00,10/17/2019,TREATABLE REHABILITABLE NON-CONTAGIOUS,EMERGENCY RESCUE,TRANSFER,MEDICAL-CONTAGIOUS,10/15/2019,17:35:00,NaN,K19-483073,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,SWEEP,TAGGED,OCT.2019,FY2020
4,A0565586,DOG,SILKY TERRIER,LFD 119,UNAVAILABLE,NaN,NaN,1,P0890077,006900,1,STRAY,AT LARGE,1,OTHRINTAKS,JR,11/08/2019,11:55:00,11/14/2019,APP WNL,RESCU ONLY,RETURNED TO OWNER,WALK IN,11/09/2019,12:57:00,R19-558750,K19-486694,NaN,APP WNL,SCAN CHIP,OVER THE COUNTER,RETURNED TO OWNER,NOV.2019,FY2020


## Data Cleaning

In [4]:
df.columns = map(str.lower, df.columns)

In [5]:
df.columns = df.columns.str.replace(' ', '_')

In [6]:
df = df.drop(['tag_type', 'activity_number', 'activity_sequence', 'source_id', 'census_tract', 
              'intake_total', 'receipt_number', 'impound_number', 'service_request_number', 
              'additional_information', 'council_district', 'hold_request', 'intake_time',
              'outcome_time'], axis=1)


In [7]:
# for the reason column, we want to combine both Other values and turn the NANS into Unknown
df.reason = df.reason.apply(lambda x: x if (x == 'PERSNLISSU' or
                                            x == 'MEDICAL' or 
                                            x == 'HOUSING' or
                                            x == 'NOTRIGHTFT' or
                                            x == 'FINANCIAL' or
                                            x == 'OTHRINTAKS' or 
                                            x == 'OTHER')
                            else 'UNKNOWN')

In [8]:
df.outcome_condition = df.outcome_condition.fillna('UNKNOWN')

In [9]:
df.chip_status = df.chip_status.fillna('UNKNOWN')

In [10]:
df.animal_origin = df.animal_origin.fillna('UNKNOWN')

In [11]:
df.dropna(subset = ['outcome_date'], inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31695 entries, 0 to 31917
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   animal_id          31695 non-null  object
 1   animal_type        31695 non-null  object
 2   animal_breed       31695 non-null  object
 3   kennel_number      31695 non-null  object
 4   kennel_status      31695 non-null  object
 5   intake_type        31695 non-null  object
 6   intake_subtype     31695 non-null  object
 7   reason             31695 non-null  object
 8   staff_id           31695 non-null  object
 9   intake_date        31695 non-null  object
 10  due_out            31695 non-null  object
 11  intake_condition   31695 non-null  object
 12  outcome_type       31695 non-null  object
 13  outcome_subtype    31695 non-null  object
 14  outcome_date       31695 non-null  object
 15  outcome_condition  31695 non-null  object
 16  chip_status        31695 non-null  objec

## EDA

In [15]:
df.kennel_status.value_counts()

UNAVAILABLE                 17239
IMPOUNDED                    8931
AVAILABLE                    3739
LAB                           953
RELEASED FROM QUARANTINE      366
EU RISK                       210
PENDING                       142
WILDLIFE                       97
PRE-LAB                        14
LIVESTOCK                       3
EVALUATION                      1
Name: kennel_status, dtype: int64

In [16]:
df.animal_breed.value_counts()

DOMESTIC SH        7202
PIT BULL           5507
GERM SHEPHERD      2990
LABRADOR RETR      2611
CHIHUAHUA SH       2291
DOMESTIC MH         603
ALASKAN HUSKY       533
CAIRN TERRIER       466
ROTTWEILER          423
OPOSSUM             394
AUST CATTLE DOG     370
SIBERIAN HUSKY      357
BOXER               344
SHIH TZU            332
GREAT PYRENEES      267
AM PIT BULL TER     263
CHIHUAHUA LH        252
RACCOON             232
YORKSHIRE TERR      230
MALTESE             223
BORDER COLLIE       208
POODLE MIN          186
SCHNAUZER MIN       179
DACHSHUND           178
DOBERMAN PINSCH     177
AUST SHEPHERD       173
AMER BULLDOG        155
BEAGLE              154
AMERICAN STAFF      143
DOMESTIC LH         129
POMERANIAN          123
CATAHOULA           119
STAFFORDSHIRE       119
RAT TERRIER         118
CHOW CHOW           109
GOLDEN RETR         107
SIAMESE              97
PARSON RUSS TER      90
RABBIT SH            84
PUG                  83
AMER SH              80
BULL TERRIER    

https://askwonder.com/research/provide-list-kpis-animal-services-departments-within-municipalities-9yyy09551

2. Percentage of animals adopted or returned to owner (higher is better—fewer animals euthanized or died)
3. Percentage of animals not euthanized (lower is better) 
4. Live release rate (higher is better) 
11. Percentage increase in number of dogs microchipped 
12. Percentage increase in number of dogs and cats returned to owners YOY
28. Number of impounds YOY
29. Live/Save rate (the number of live releases divided by the number of live impounds)
31. Number of spay/neuter services performed
32. Number of animals euthanized 
33. Number of animals adopted 
34. Number of animals redeemed by owner
35. Number of animals died in shelter 
39. Number of transfers/fosters 
40. Euthanasia rates YOY
41. Adoption rates YOY
43. Return to owner rates YOY

In [15]:
df.head()

,animal_id,animal_type,animal_breed,kennel_number,kennel_status,intake_type,intake_subtype,reason,staff_id,intake_date,due_out,intake_condition,outcome_type,outcome_subtype,outcome_date,outcome_condition,chip_status,animal_origin,month,year
0,A0144701,DOG,HAVANESE,VT 12,IMPOUNDED,OWNER SURRENDER,GENERAL,PERSNLISSU,CDM,11/08/2019,11/14/2019,APP SICK,RETURNED TO OWNER,WALK IN,11/09/2019,APP SICK,SCAN CHIP,OVER THE COUNTER,NOV.2019,FY2020
1,A0442587,DOG,TERRIER MIX,FREEZER,IMPOUNDED,OWNER SURRENDER,DEAD ON ARRIVAL,OTHRINTAKS,CDM,11/10/2019,11/10/2019,DEAD,DEAD ON ARRIVAL,DISPOSAL,11/10/2019,DEAD,SCAN CHIP,OVER THE COUNTER,NOV.2019,FY2020
2,A0458972,DOG,CATAHOULA,RECEIVING,UNAVAILABLE,STRAY,AT LARGE,OTHER,MG1718,10/03/2019,10/03/2019,TREATABLE REHABILITABLE NON-CONTAGIOUS,RETURNED TO OWNER,FIELD,10/03/2019,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN NO CHIP,SWEEP,OCT.2019,FY2020
3,A0525642,DOG,GERM SHEPHERD,INJD 001,IMPOUNDED,OWNER SURRENDER,GENERAL,OTHER,RA 1549,10/11/2019,10/17/2019,TREATABLE REHABILITABLE NON-CONTAGIOUS,TRANSFER,MEDICAL-CONTAGIOUS,10/15/2019,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,SWEEP,OCT.2019,FY2020
4,A0565586,DOG,SILKY TERRIER,LFD 119,UNAVAILABLE,STRAY,AT LARGE,OTHRINTAKS,JR,11/08/2019,11/14/2019,APP WNL,RETURNED TO OWNER,WALK IN,11/09/2019,APP WNL,SCAN CHIP,OVER THE COUNTER,NOV.2019,FY2020


In [17]:
df.month.value_counts()

OCT.2019    3996
JAN.2020    3659
NOV.2019    3526
DEC.2019    3367
FEB.2020    3127
MAR.2020    2731
JUN.2020    2127
MAY.2020    2039
JUL.2020    2007
AUG.2020    1796
APR.2020    1691
SEP.2020    1629
Name: month, dtype: int64